In [ ]:
'''
📝 Exercício 1: Listar todos os alunos ativos com idade maior que 20 anos, ordenados por nome.
mongosh: Use find() com filtros e sort()
Python: Implemente com PyMongo usando filtros e ordenação
'''
from pymongo import MongoClient

# Conexão com o MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["test"]
alunos = db["alunos"]

# Filtro e ordenação
resultado = alunos.find(
    {"ativo": True, "idade": {"$gt": 20}}
).sort("nome", 1)

# Exibir resultados
for aluno in resultado:
    print(aluno)
    
"---------------------------------------------------------------------------------------------------"
'''
db.alunos.find(
  { ativo: true, idade: { $gt: 20 } }
).sort({ nome: 1 })
'''

{'_id': ObjectId('68d6e45af80733e98159eb51'), 'nome': 'Bruno Santos', 'idade': 22, 'curso': 'Medicina', 'notas': [9.5, 8, 9], 'cidade': 'Rio de Janeiro', 'ativo': True, 'data_matricula': datetime.datetime(2023, 2, 1, 0, 0)}
{'_id': ObjectId('68d6e45af80733e98159eb53'), 'nome': 'Diana Costa', 'idade': 25, 'curso': 'Direito', 'notas': [8, 9, 8.5], 'cidade': 'Belo Horizonte', 'ativo': True, 'data_matricula': datetime.datetime(2022, 8, 20, 0, 0)}
{'_id': ObjectId('68d6e45af80733e98159eb54'), 'nome': 'Eduardo Rocha', 'idade': 21, 'curso': 'Computação', 'notas': [9, 9.5, 8.5], 'cidade': 'São Paulo', 'ativo': True, 'data_matricula': datetime.datetime(2023, 4, 12, 0, 0)}


In [ ]:
'''
📝 Exercício 2: Contar quantos alunos existem por curso e mostrar apenas cursos com mais de 1 aluno.
mongosh: Use aggregate() com $group e $match
Python: Crie uma função que retorne um dicionário com o resultado
'''
from pymongo import MongoClient

def contar_alunos_por_curso():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["test"]
    alunos = db["alunos"]

    pipeline = [
        {"$group": {"_id": "$curso", "totalAlunos": {"$sum": 1}}},
        {"$match": {"totalAlunos": {"$gt": 1}}}
    ]

    resultado = alunos.aggregate(pipeline)

    # Converter para dicionário {curso: totalAlunos}
    return {doc["_id"]: doc["totalAlunos"] for doc in resultado}

# Exemplo de uso
print(contar_alunos_por_curso())

"-------------------------------------------------------------------------------------------------------"
'''
db.alunos.aggregate([
  { $group: { _id: "$curso", totalAlunos: { $sum: 1 } } },
  { $match: { totalAlunos: { $gt: 1 } } }
])

'''

{'Engenharia': 2}


In [ ]:
'''
📝 Exercício 3: Encontrar todos os alunos cujo nome contém "Silva" (case-insensitive).
mongosh: Use $regex com $options
Python: Implemente busca por regex usando re (módulo Python)
'''

from pymongo import MongoClient
import re

# Conexão com MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["test"]
alunos = db["alunos"]

# Regex em Python (case-insensitive)
regex = re.compile("Silva", re.IGNORECASE)

resultado = alunos.find({"nome": regex})

# Exibir resultados
for aluno in resultado:
    print(aluno)
    
"-------------------------------------------------------------------------------------------------------"
'''
db.alunos.find(
  { nome: { $regex: "Silva", $options: "i" } }
)

'''

{'_id': ObjectId('68d6e45af80733e98159eb50'), 'nome': 'Ana Silva', 'idade': 20, 'curso': 'Engenharia', 'notas': [8.5, 9, 7.5], 'cidade': 'São Paulo', 'ativo': True, 'data_matricula': datetime.datetime(2023, 3, 15, 0, 0)}


In [ ]:
'''
📝 Exercício 4: Calcular a média geral de notas de todos os alunos ativos.
mongosh: Use aggregate() com $unwind e $group
Python: Processe os dados e calcule a média usando statistics.mean()
'''
from pymongo import MongoClient
from statistics import mean

client = MongoClient("mongodb://localhost:27017/")
db = client["test"]
alunos = db["alunos"]

# Buscar alunos ativos
ativos = alunos.find({"ativo": True})

# Extrair todas as notas
todas_notas = []
for aluno in ativos:
    todas_notas.extend(aluno.get("notas", []))  # garante que, se não houver notas, não quebre

# Calcular média geral
if todas_notas:
    media_geral = mean(todas_notas)
    print(f"Média geral: {media_geral:.2f}")
else:
    print("Não há notas para calcular a média.")
    
"-------------------------------------------------------------------------------------------------------"
'''
db.alunos.aggregate([
  { $match: { ativo: true } },           // filtrar alunos ativos
  { $unwind: "$notas" },                 // "desempacotar" as notas do array
  { $group: { 
      _id: null, 
      mediaGeral: { $avg: "$notas" }    // calcular média de todas as notas
  }}
])

'''

Média geral: 8.67


In [ ]:
'''
📝 Exercício 5: Listar os 3 alunos com maior média individual, mostrando nome, curso e média.
mongosh: Use $addFields para calcular média, depois $sort e $limit
Python: Calcule médias em memória e use sorted() com key personalizada
'''
from pymongo import MongoClient
from statistics import mean

client = MongoClient("mongodb://localhost:27017/")
db = client["test"]
alunos = db["alunos"]

# Buscar todos os alunos
todos_alunos = list(alunos.find())

# Calcular média individual
for aluno in todos_alunos:
    notas = aluno.get("notas", [])
    aluno["media"] = mean(notas) if notas else 0

# Ordenar e pegar top 3
top3 = sorted(todos_alunos, key=lambda x: x["media"], reverse=True)[:3]

# Exibir resultados
for aluno in top3:
    print(f"Nome: {aluno['nome']}, Curso: {aluno['curso']}, Média: {aluno['media']:.2f}")

"-------------------------------------------------------------------------------------------------------"
'''
db.alunos.aggregate([
  { 
    $addFields: { media: { $avg: "$notas" } }  // calcula a média individual
  },
  { $sort: { media: -1 } },                    // ordena da maior para a menor
  { $limit: 3 },                               // pega apenas os 3 primeiros
  { $project: { nome: 1, curso: 1, media: 1, _id: 0 } }  // mostra apenas os campos desejados
])

'''

Nome: Eduardo Rocha, Curso: Computação, Média: 9.00
Nome: Bruno Santos, Curso: Medicina, Média: 8.83
Nome: Diana Costa, Curso: Direito, Média: 8.50
